#### 导库

In [2]:
import asyncio
import aiohttp
import time
import random
import requests
import pymongo
from lxml import etree
from parsel import Selector

#### MongoDB 连接

In [3]:
client = pymongo.MongoClient(host='localhost', port=27017)
db = client['indianhospital']

province_col          = db['province']
hospital_col          = db['hospital']
province_all_href_col = db['province_all_href']
hospital_detail_info  = db['hospital_detail_info']

# error_province_col = db['error_province']
# error_hospital_col = db['error_hospital']

#### 请求头 & 随机代理

In [4]:
headers = {
    'sec-ch-ua' : '"Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105"',
    'sec-ch-ua-mobile'  : '?0',
    'sec-ch-ua-platform': '"Windows"',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
}

def random_proxies():
    proxies = {
        'http' : 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()
    }
    return proxies

def random_proxy():
    proxy = 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()
    return proxy

random_proxy()

'http://85.14.243.31:3128'

#### 获取 37 个省的信息

- 包含：名字、链接

-   

In [8]:
"""
一、获取 37 个省份的链接
返回一个包含 37 个省份链接的列表
"""
origin_url = 'https://www.medindia.net/patients/hospital_search/hospital_list.asp?utm_source=topnavigation&utm_medium=desktop&utm_content=&utm_campaign=medindia'
try:
    origin_res = requests.get(
        url     = origin_url,
        headers = headers,
        proxies = random_proxies()
    )
    selector = Selector(text = origin_res.text)

    for div in selector.xpath('//ul[@class="list-inline"]/div/div'):
        province_name = div.xpath('./li/a/span/text()').get().title()   # 所有单词首字母大写
        province_href = div.xpath('./li/a/@href').get()
        if province_col.find_one({'province_name':province_name}) != None:
            continue
        else:
            province_col.insert_one({
                'province_name'  :province_name,
                'province_href'  :province_href,
                'province_status':None
            })
    print('【SUCCEED -- CRAWL PROVINCE SUCCEED】')
except:
    print('【ERROR   -- CRAWL PROVINCE FAILED 】')

【SUCCEED -- CRAWL PROVINCE SUCCEED】


#### 采集医院包含的所有页

In [5]:
province_status_list = []

for province in province_col.find():
    province_name = province['province_name']
    province_href = province['province_href']
    # 先判断是否存在某省的医院
    try:
        selector = Selector(text = requests.get(province_href, headers=headers, proxies=random_proxies()).text)
        records  = selector.xpath('//p[@class="searchtext"][contains(text(), "Total Records")]/b[1]/text()').get()
        page     = selector.xpath('//p[@class="searchtext"][contains(text(), "Total Records")]/b[last()]/text()').get()
        
        if records == page:
            province_total_page = 1
        else:
            province_total_page = page
        province_total_href = [province_href[:-4] + f'-{str(i)}' + province_href[-4:] for i in range(1, province_total_page+1)]

        for href in province_total_href:
            province_all_href_col.insert_one({
                'province_name'          : province_name,
                'province_all_page_href' : href
            })
        province_status_list.append({'province_name':province_name,'province_new_status':1})
    except:
        province_status_list.append({'province_name':province_name,'province_new_status':0})

#### 更新 **`province_col`** 数据库的状态

In [ ]:
for province in province_status_list:
    condition        = {'province_name':province['province_name']}
    update_status    = province_col.find_one(condition)
    update_status['province_status']  = province['province_new_status']
    
    province_col.update_one(condition,{'$set': update_status})

#### 获取每一个省包含的所有医院信息

In [29]:
province_status_list = []
for province in province_col.find()[9:10]:
    province_name = province['province_name']
    province_href = province['province_href']
    # 先判断是否存在某省的医院
    if hospital_col.find_one({'province_name':province_name}) != None:
        continue
    else:
        try:
            province_res  = requests.get(
                url     = province_href,
                headers = headers,
                proxies = random_proxies(),
                verify  = False,
            )
            selector = Selector(text = province_res.text)

            hospital_list = selector.xpath('//h3[@class="vert-small-margin"]/a')
            for hospital in hospital_list:
                hospital_name = hospital.xpath('./text()').get().title()
                hospital_href = hospital.xpath('./@href').get()
                # 判断医院是否已经存入数据库中
                if hospital_col.find_one({'hospital_name':hospital_name}) != None:
                    continue
                else:
                    hospital_col.insert_one({
                        'province_name'  :province_name,
                        'hospital_name'  :hospital_name,
                        'hospital_href'  :hospital_href,
                        'hospital_status':None
                    })
            print(f'【SUCCDDE -- GET ALL HOSPITAL OF {province_name} SUCCEED】')
            province_status_list.append({'province_name':province_name,'province_new_status':1})
        except:
            print(f'【ERROR   -- GET ALL HOSPITAL OF {province_name} FAILED 】')
            province_status_list.append({'province_name':province_name, 'province_new_status':0})

【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【ERROR -- GET ALL HOSPITAL OF PROVINCE FAILED
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【ERROR -- GET ALL HOSPITAL OF PROVINCE FAILED
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【ERROR -- GET ALL HOSPITAL OF PROVINCE FAILED
【SUCCDDE -- GET ALL HOSPITAL OF PROVINCE SUCCEED】
【SUCCDDE -- 

#### 采集所有医院的详细信息

In [ ]:
hospital_name = selector.xpath('//div[@class="mi-bg-1"]/../h2/text()').re_first('Address of (.*)')
hospital_address       = ', '.join(selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p[1]//text()').re('\s*(\w.*\w)\s*,*'))
hospital_director      = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Director")]/../text()').re_first('\s*(\w.*\w)\s*')
hospital_email         = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Email")]/../span/text()').re_first('\s*(\w.*\w)\s*')
hospital_phone         = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Phone")]/../span/text()').re_first('\s*(\w.*\w)\s*')
hospital_mobile        = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Mobile")]/../span/text()').re_first('\s*(\w.*\s)\s*')

hospital_detail_info = {
    'hospital_name'    : hospital_name,
    'hospital_address' : hospital_address,
    'hospital_director': hospital_director,
    'hospital_email'   : hospital_email,
    'hospital_phone'   : hospital_phone,
    'hospital_mobile'  : hospital_mobile,
    'hospital_href' : None,
    'hospital_html' : None,
}